# Pronósticos Usando Deep Learning
## ¡Un Pronóstico Probabilístico!

¡Hola de nuevo! Anteriormente realizamos un ejercicio para pronosticar el consumo de energía eléctrica de 370 consumidores en USA. Nuestro problema sigue siendo el mismo, es decir:

![context](imgs/context.png)

Sin embargo, a lo largo de este ejercicio daremos un paso más allá. Y es que nuestro pronóstico no será únicamente un pronóstico puntual, sino que generaremos toda una distribución al rededor de sus posibles valores.

Este ejercicio será guiado, pero dependerá de ti resolverlo y experimentar. 

Completa el código donde veas la indicación `### TU CÓDIGO AQUÍ ###`

<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/juancop/URDeepLearning/blob/main/Forecasting/%5BHANDS%20ON%5D%20Pronóstico%20Probabilístico.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/juancop/URDeepLearning/blob/main/Forecasting/Pron%C3%B3stico%20Probabil%C3%ADstico.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  
</table>

Si estás en colab corre la siguiente celda:

In [ ]:
!git clone https://github.com/juancop/URDeepLearning && mv URDeepLearning/Forecasting/sample .

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import tensorflow as tf
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [ ]:
np.random.seed(42)

In [ ]:
tf.__version__ # ¡Verifica nuevamente que estés usando la versión 2.3.1!

## Carga de Información

In [ ]:
df = pd.read_csv('sample/electricity.csv')

In [ ]:
df.head()

# Datos para Entrenamiento

En este punto estmaos ya familiarizados con la información que estamos trabajando, por lo que nos saltaremos la exploración de los datos. 

La generación de ventanas de entrenamiento es bastante importante, pues a partir de ellas el modelo aprende. En esta sección te encargarás de crear una función de procesamiento que te entregue la variable objetivo como la desees (ya sea con o sin logaritmo) y el conjunto de variables explicativas que quieras usar. 

In [ ]:
class processElectricityDataFrame:
    """
    Procesa el DataFrame para generar un data set de ventanas.
    """
    
    def __init__(self, train_df, val_df, test_df, index_variable, target_variable,
                input_window, output_window, step = 24):
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df   
        
        self.label_name = target_variable
        self.input_window = input_window
        self.output_window = output_window
        self.index_variable = index_variable
        self.step = step
        
        
        self.model_columns = None 
            
    
    def create_npy_dataset(self, df, folder, name, columns, input_window, output_window, label_name):
        """
            df: DataFrame a utilizar
            folder: Folder para almacenar el .npy
            columns: Lista de las columnas que se utilizarán al final
            name: Nombre para el archivo (train, val, test)
            input_window: Tamaño de la ventana de entrada (12)
            output_window: Tramaño de la ventana de salida (12)
            label_name: Nombre de la variable objetivo
        
        """
        file_input = os.path.join(folder, f'input_{name}.npy')
        file_output = os.path.join(folder, f'output_{name}.npy')

        unique_id_names = df[self.index_variable].value_counts().index.values
        npy_input = None
        npy_output = None

        label_index = columns.index(label_name)
        input_columns = columns.copy()
            
        window_size = input_window + output_window
        
        print(f'Started .npy creation with name: {name}')
        for id_name in tqdm(unique_id_names):
            
            customer_df = df[df[self.index_variable] == id_name].sort_values(by = 'date').copy()
            output_tmp_df = customer_df[columns].values
            input_tmp_df = customer_df[input_columns].values
            n_windows = len(output_tmp_df) - window_size + 1
            
            input_array = np.expand_dims(np.arange(self.input_window), 0) +  np.expand_dims(np.arange(n_windows, step = self.step), 0).T
            output_array = np.expand_dims(np.arange(self.output_window), 0) + self.input_window +  np.expand_dims(np.arange(n_windows, step = self.step), 0).T
            
            
            inputs = input_tmp_df[input_array].astype(np.float32)
            labels = output_tmp_df[output_array, label_index].astype(np.float32)
            
            if npy_input is None:
                
                npy_input = inputs
                npy_output = labels
            else:
                
                npy_input = np.append(npy_input, inputs, axis = 0)
                npy_output = np.append(npy_output, labels, axis = 0)
                
                random_index = np.random.choice(npy_input.shape[0], npy_input.shape[0], replace=False)
                
                npy_input = npy_input[random_index, :, :]
                npy_output = npy_output[random_index, :]

        with open(file_input, 'wb') as f:
            np.save(f, npy_input)

        with open(file_output, 'wb') as f:
            np.save(f, npy_output)

        print('Completed .npy creation.')        

        
    def preprocess_fn(self, df, model_type):      
        if model_type == 'raw_inf':
            df[self.label_name] = np.log(df[self.label_name] + 1)
            self.model_columns = [self.label_name]#, 'hour', 'day_of_week', 'month']
            return df
        
        
        ### TU CÓDIGO AQUÍ ####
        
        # Define tu función de procesamiento e indica en self.model_columns las variables que utilizarás para entrenar
        # OJO: La variable objetivo debe ser la primera en la lista.
        
        # Esta función debe retornar un data frame con todas las variables que definiste
        
        if model_type == 
        
        ### TERMINA ###

        
    def create_all_datasets(self, folder_name = 'data', model_type = 'raw_inf'):
        
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
        
        train_df = self.preprocess_fn(self.train_df, model_type)    
         
        self.create_npy_dataset(df = train_df, folder = folder_name, name = 'train', columns =  self.model_columns,
                               input_window = self.input_window, output_window = self.output_window, label_name = self.label_name)
        
        val_df = self.preprocess_fn(self.val_df, model_type)    
         
        self.create_npy_dataset(df = val_df, folder = folder_name, name = 'val', columns =  self.model_columns,
                               input_window = self.input_window, output_window = self.output_window, label_name = self.label_name)
        
        
        test_df = self.preprocess_fn(self.test_df, model_type)    
         
        self.create_npy_dataset(df = test_df, folder = folder_name, name = 'test', columns =  self.model_columns,
                               input_window = self.input_window, output_window = self.output_window, label_name = self.label_name)

In [ ]:
test_df = df[df.date >= '2014-08-31']
val_df = df[(df.date >= '2014-08-24') & (df.date < '2014-08-31')]
train_df = df[df.date < '2014-08-24']


A continuación generaremos los datos para el modelo. ¡Sí! Con las ventanas y la función de procesamiento que definiste.

In [ ]:
df_process = processElectricityDataFrame(train_df, 
                                         val_df, 
                                         test_df, 
                                         index_variable = 'categorical_id',
                                         target_variable = 'power_usage',
                                         input_window = 48,
                                         output_window = 24,
                                         step = 8)

In [ ]:
### TU CÓDIGO AQUÍ ###
#¿Cómo llamaste a tu función de procesamiento? 
df_process.create_all_datasets(folder_name = 'data', model_type = )

### TERMINA ###



# Pronóstico Probabilístico

Hasta este punto no hemos hecho nada raro. ¿Cómo hacemos que nuestro modelo prediga un cuantil si ni siquiera conocemos la distribución subyacente a cada serie? La respuesta se encuentra en lo que se conoce como *Quantile Loss*, o también *Pinball Loss*.  ¿Te suena la métrica conocida como $\operatorname{MAD}$? En el contexto de regressión lineal, si el criterio es minimizar la $\operatorname{MAD}$ (Median Absolute Deviation), estaríamos haciendo una regresión sobre la *mediana* en lugar de la media. 

Esta función de pérdida viene dada por, 

$$QL(y, \hat{y}, \tau) = \operatorname{max}(\tau (y - \hat{y}), (1 - \tau)(\hat{y} - y))$$

A modo de ejemplo, si $\tau = 0.75$, la función se vería como:

$$QL(y, \hat{y}, \tau) = \operatorname{max}(0.75 (y - \hat{y}), 0.25(\hat{y} - y))$$

donde el modelo penalizaría más fuertemente una predicción por encima de $y$ que una por debajo. Lo que se busca es que los errores de predicción por encima sean apenas el 25% de las veces, mientras que por debajo sean el 75% de las veces. 

Dicho lo anterior, completa la función de pérdida para nuestro problema, asumiendo que `y_true` es un vector de dimensión `(N, T)` y `y_pred`es de dimensión `(N, T, q)`, donde: 
1. `N` Es la cantidad de individuos
2. `T` Es la cantidad de puntos en el tiempo (el horizonte de pronóstico)
3. `q` corresponde a la cantidad de cuantiles estimados


Utiliza las siguientes funciones para completar el código: 
- ``tf.math.subtract``: https://www.tensorflow.org/api_docs/python/tf/math/subtract
- ``tf.math.multiply``: https://www.tensorflow.org/api_docs/python/tf/math/multiply
- ``tf.math.maximum``: https://www.tensorflow.org/api_docs/python/tf/math/maximum


In [ ]:
from tensorflow.keras.losses import Loss

class QuantileLoss(Loss):
    
    def __init__(self, percentiles = [0.1, 0.5, 0.9]):
        super().__init__()
        self.percentiles = percentiles
        self.n_quantiles = len(self.percentiles)

    def call(self, y_true, y_pred):
        """
        Computes the Quantile Loss
        """
        
        percentiles = tf.constant(self.percentiles, shape = [1, self.n_quantiles], dtype = float)
        
        # Redimensionamos el vector objetivo para repetir sus columnas por cada cuantil a estimar
        y_true = tf.expand_dims(y_true, -1)
        y_true = tf.repeat(y_true, repeats = self.n_quantiles, axis=-1)
        
        
        ### TU CÓDIGO ###
        # Calcula el error de predicción (y - yhat)
        error = 
        
        ### TERMINA ###
        
        under_prediction_error = tf.math.multiply(percentiles, error) # Cálculo de primer término
        
        ### TU CÓDIGO ###
        # Calcula el segundo término de error
        over_prediction_error = 
        
        ### TERMINA ###
        
        
        ### TU CÓDIGO ###
        # Calcula la pérdida utilizando el máximo
        Quantile_Loss = 
        
        ### TERMINA ###
        
        return tf.math.reduce_mean(Quantile_Loss)

¡Verifica tu resultado!

Debes obtener:

`<tf.Tensor: shape=(), dtype=float32, numpy=1.2333332>`


In [ ]:
y_true = tf.constant([1, 2, 3], dtype = float)
y_pred =  tf.constant([4, 5, 6], dtype = float)

QL = QuantileLoss()
QL.call(y_true, y_pred)

# Entrenamiento de Modelo

In [ ]:
def load_dataset(path_to_features, path_to_labels):
    features = np.load(path_to_features)
    labels = np.load(path_to_labels)
    
    assert features.shape[0] == labels.shape[0]
    dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(256)
    return dataset

In [ ]:
train_features = os.path.join('data', 'input_train.npy')
train_labels = os.path.join('data', 'output_train.npy')
train_dataset = load_dataset(train_features, train_labels)

val_features = os.path.join('data', 'input_val.npy')
val_labels = os.path.join('data', 'output_val.npy')
val_dataset = load_dataset(val_features, val_labels)

test_features = os.path.join('data', 'input_test.npy')
test_labels = os.path.join('data', 'output_test.npy')
test_dataset = load_dataset(test_features, test_labels)

In [ ]:
def validate_inf(arrays):
    for array in arrays:
        loaded_array = np.load(os.path.join(array))
        n_inf = np.isinf(loaded_array).sum()
        if n_inf:
            print(f'Array {array} contains INF values')
            
        n_nan = np.isnan(loaded_array).sum()
        if n_nan:
            print(f'Array {array} contains NAN values')

¡Listo! Podremos realizar el entrenamiento de nuestro modelo. Para poderlo hacer necesitarás definir dos cosas:

1. ¿Qué percentiles utilizarás? Por defecto tenemos el percentil 10%, 50% y 90%, pero puedes definir otros diferentes.
2. ¡Claramente la arquitectura de red!

In [ ]:
OUT_STEPS = 24

### TU CÓDIGO AQUÍ ###

# Define los percentiles que quieres estimar en una lista (e.g. [0.1, 0.5, 0.9])
PERCENTILES = 

### TERMINA ###

percentiles = len(PERCENTILES)


model = tf.keras.models.Sequential([
                ### TU CÓDIGO AQUÍ ###
                # Define la arquitectura de red que quieras utilizar, no definas la capa de salida
    
              
                ### TERMINA ###
                tf.keras.layers.Dense(OUT_STEPS*percentiles, kernel_initializer=tf.initializers.zeros()),
                tf.keras.layers.Reshape((OUT_STEPS, percentiles))
                ])

In [ ]:

checkpoint_folder = 'training/'
tf_log_dir = 'training/logs'

if not os.path.exists(checkpoint_folder):
    os.mkdir(checkpoint_folder)
    
if not os.path.exists(tf_log_dir):
    os.mkdir(tf_log_dir)


In [ ]:
name = 'pfcst'
execution_time = datetime.now().strftime("%Y%m%d-%H%M%S")
tf_log_dir = 'training'
logdir = f"{tf_log_dir}/probabilistic/{name}_{execution_time}" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_path = f"{checkpoint_folder}/checkpoints_{name}_{execution_time}/cp.ckpt"
checkpoints_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)

### TU CÓDIGO AQUÍ
# Define tu Early Stopping Callback

earlystopping_callback = 

### TERMINA

In [ ]:
### TU CÓDIGO AQUÍ
# Define el otpimizador que vas a utilizar y sus parámetros
optimizer = 

### TERMINA 

EPOCHS = 50

model.compile(
      optimizer=optimizer,
      loss=QuantileLoss(PERCENTILES),# tf.losses.MeanSquaredError(),
     
    )

In [ ]:
model.fit(train_dataset, 
          validation_data=val_dataset, 
          callbacks=[tensorboard_callback, checkpoints_callback, earlystopping_callback],
          epochs=EPOCHS, 
          verbose =1) 

Para este punto ya tenemos un modelo entrenado, y el mejor dada la configuración se encuentra almacenado en `checkpoint_path`. A continuación vamos a observar sus resultados y hacer una pequeña evaluación. 

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.evaluate(val_dataset)

# Análisis de Desempeño

Carga los datos de prueba y realiza la predicción del modelo.

In [ ]:
X = np.load(test_features)
Y = np.load(test_labels)

In [ ]:
### TU CÓDIGO AQUÍ
# Realiza la predicción del modelo
Y_hat = 

### TERMINA

A continuación, deberás definir la función que graficará tus resultados. Ten en cuenta que debes realizar las transformaciones inversas a las que aplicaste en la generación de ventanas.

In [ ]:
def plot_predictions(Y_past, Y_true, Y_pred, idx_obs, exponent = True):
    plt.figure(figsize = (10, 5))
    input_length = Y_past.shape[1]
    ouput_length = Y_pred.shape[1]

    plt.plot(np.arange(input_length), Y_past[idx_obs, :, 0], color = 'black')

    ### TU CÓDIGO AQUÍ ###
    # Grafica la serie objetivo y la serie predicha para un idx_obs particular

    
    
    ### TERMINA ###

    
    # Asumimos que el cuantil más bajo es el primero y el más alto es el último
    plt.fill_between(np.arange(input_length, input_length+ouput_length), Y_pred[idx_obs, :, 0], Y_pred[idx_obs, :, -1], color='b', alpha=.1)
    plt.legend()

In [ ]:
plot_predictions(X, Y, Y_hat, 3, False)

Ya que tenemos una función para graficar nuestras predicciones, vamos a hacer el cálculo del MAPE de cada serie y del modelo en general. 

In [ ]:
def compute_mape(Y_true, Y_pred, return_list = True):

    Y_true = Y_true + 1
    ### TU CÓDIGO AQUÍ ###
    # Escoge el quantil sobre el cual vas a hacer la comparación (principalmente sería la mediana)
    Y_pred = Y_pred[:, :, 1]
        
    ### TERMINA
    
    Y_pred = Y_pred.clip(min = 1)
    
    ### TU CÓDIGO AQUÍ ###
    # Calcula el MAPE para cada serie
    
    
    MAPE_customer =  # Una lista de MAPE de dimensión Y_true.shape[0]
    
    ### TERMINA ###
    print('Overall model MAPE: {:.2%}'.format(np.mean(MAPE_customer)))
    print('Overall model MAPE (median): {:.2%}'.format(np.median(MAPE_customer)))
    return MAPE_customer
    

In [ ]:
def plot_ranked(Y_true, Y_pred, X, mape, best = True, top = 5, exponent = True):
    sorted_mape = np.argsort(mape)
    if best:
        idx = sorted_mape[:top]
    else:
        idx = sorted_mape[-top:]
    for i in idx:
  
        plot_predictions(X, Y_true, Y_pred, i, exponent)
        plt.title("{:.2%}".format(mape[i]))
        plt.plot()

In [ ]:
mape_list = compute_mape(Y, Y_hat, exponent = False)

In [ ]:
plot_ranked(Y, Y_hat, X, mape_list, exponent = False)

In [ ]:
plot_ranked(Y, Y_hat, X, mape_list, exponent = False, best = False)